In [1]:
!pip install requests
!pip install opencv-python
!pip install googledrivedownloader
from google_drive_downloader import GoogleDriveDownloader as gdd

gdd.download_file_from_google_drive(file_id='0BxYys69jI14kYVM3aVhKS1VhRUk',
                                    dest_path='./gender/UTKFace.tar.gz',
                                    unzip=False)

You are using pip version 18.0, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 18.0, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 18.0, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
%cd gender
import tarfile
tar = tarfile.open("/notebooks/gender/UTKFace.tar.gz")
tar.extractall()
tar.close()

/notebooks/gender


In [3]:
!pip install tqdm
import pandas as pd
import numpy as np
import os
from tqdm import tqdm
import cv2
from sklearn.preprocessing import OneHotEncoder
class images_to_array():
    def __init__(self,imagesize,directory):
        self.imagesize = imagesize
        self.directory = directory
    def extract(self):
        a = os.listdir(self.directory)
        image = np.ones((23708,3*self.imagesize,self.imagesize),int)
        image = np.reshape(image,(23708,self.imagesize,self.imagesize,3))
        for i in tqdm(range(len(a))):
          path = os.path.join(self.directory,str(a[i]))
          image1 = cv2.imread(path)
          image1 = cv2.resize(image1,(self.imagesize,self.imagesize))
          image1 = np.reshape(image1,(self.imagesize,self.imagesize,3))
          image[i,:,:,:] = image1
        return image

You are using pip version 18.0, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [4]:
imagesize = 224
directory = "/notebooks/gender/UTKFace"
obj = images_to_array(imagesize,directory)
img = obj.extract()

100%|██████████| 23708/23708 [00:40<00:00, 585.85it/s]


In [5]:
!mkdir git
%cd git
!git clone https://github.com/sanchit2843/agedetect
%cd agedetect
!cp gender.csv /notebooks/gender
%cd /notebooks/gender
!rm -r git
!rm UTKFace.tar.gz
!ls

/notebooks/gender/git
Cloning into 'agedetect'...
remote: Enumerating objects: 70, done.
remote: Counting objects: 100% (70/70), done.
remote: Compressing objects: 100% (58/58), done.
remote: Total 70 (delta 27), reused 45 (delta 11), pack-reused 0
Unpacking objects: 100% (70/70), done.
Checking connectivity... done.
/notebooks/gender/git/agedetect
/notebooks/gender
UTKFace  gender.csv


In [6]:
!pwd
!ls

/notebooks/gender
UTKFace  gender.csv


In [8]:
import pandas as pd
dataset = pd.read_csv('gender.csv')
y = dataset.iloc[:,1].values

In [18]:
!pip install keras
from keras import applications
from keras import optimizers
from keras.models import Sequential,Model
from keras import layers
from keras.layers import Activation,Dropout,Flatten
from keras import backend as K
import tensorflow as tf


You are using pip version 18.0, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [21]:
inception_base = applications.ResNet50(weights='imagenet',include_top=False)

x = inception_base.output
x = GlobalAveragePooling2D()(x)
x = Dense(512,activation='relu')(x)

predictions = Dense(1,activation='sigmoid')(x)

model = Model(inputs=inception_base.input,outputs=predictions)

model.compile(optimizer = optimizers.Adam(lr = 0.0001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

AttributeError: 'Node' object has no attribute 'output_masks'

In [ ]:
batch_size = 100
model.fit(x = img,y = y,batch_size = batch_size,epochs = 20)